# Import Libraries

In [1]:
from GoogleNews import GoogleNews
from newspaper import Article
from newspaper import Config
import pandas as pd
import nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/alecmccabe/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

**Set Config**

config will allow us to access the specified url for which we are #not authorized. Sometimes we may get 403 client error while parsing #the link to download the article.

In [2]:
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'
config = Config()
config.browser_user_agent = user_agent

**Set Targets by Outlet**
- outlets selected using allsides.com, a crowdsourced media political ranking system

In [3]:
far_right = ['breitbart-news','national-review']

right = ['fox-news','the-american-conservative','the-washington-times','the-wall-street-journal']

center = ['associated-press','axios','bbc-news','national-geographic','reuters','business-insider','fortune','the-hill']

left = ['abc-news','al-jazeera-english','bloomberg','business-insider','cbs-news','cnn','google-news','independent',
       'nbc-news','politico','time']

far_left = ['buzzfeed','mashable','msnbc','new-york-magazine','vice-news']

In [4]:
master_key = 'a30b2d04fd3c4d6aa0863cffa772c05b'

**NewsApi allows me to download URLs for above outlets**

In [5]:
from newsapi import NewsApiClient

# Init
master = NewsApiClient(api_key=master_key)

In [6]:
[(source['id'], source['name']) for source in master.get_sources()['sources']]

[('abc-news', 'ABC News'),
 ('abc-news-au', 'ABC News (AU)'),
 ('aftenposten', 'Aftenposten'),
 ('al-jazeera-english', 'Al Jazeera English'),
 ('ansa', 'ANSA.it'),
 ('argaam', 'Argaam'),
 ('ars-technica', 'Ars Technica'),
 ('ary-news', 'Ary News'),
 ('associated-press', 'Associated Press'),
 ('australian-financial-review', 'Australian Financial Review'),
 ('axios', 'Axios'),
 ('bbc-news', 'BBC News'),
 ('bbc-sport', 'BBC Sport'),
 ('bild', 'Bild'),
 ('blasting-news-br', 'Blasting News (BR)'),
 ('bleacher-report', 'Bleacher Report'),
 ('bloomberg', 'Bloomberg'),
 ('breitbart-news', 'Breitbart News'),
 ('business-insider', 'Business Insider'),
 ('business-insider-uk', 'Business Insider (UK)'),
 ('buzzfeed', 'Buzzfeed'),
 ('cbc-news', 'CBC News'),
 ('cbs-news', 'CBS News'),
 ('cnn', 'CNN'),
 ('cnn-es', 'CNN Spanish'),
 ('crypto-coins-news', 'Crypto Coins News'),
 ('der-tagesspiegel', 'Der Tagesspiegel'),
 ('die-zeit', 'Die Zeit'),
 ('el-mundo', 'El Mundo'),
 ('engadget', 'Engadget'),
 ('e

In [93]:
news_list = master.get_everything(q=,
                                    sources='fox-news',
                                    language='en',
                                    sort_by='relevancy',
                                    from_param= '2020-09-01',
                                    to = '2020-09-31',
                                    page_size=2,
                                    page=1)

articles = news_list['articles']

In [96]:
test = pd.DataFrame(articles)

# Begin Collecting URLs

In [8]:
def generate_news(sources, newsapi, from_date, to_date, leanings, dataframe=None):
    if type(dataframe) != type(None):
        data = dataframe
        print('provided dataframe')
    else:
        data = pd.DataFrame(columns=['source','author','title','description','url','urlToImage',
                                     'publishedAt','content','target'])
        print('no provided dataframe')
    
    for source in sources:
        try:
            print('looking for {}'.format(source))
            for i in range(1,300):
                try:
                    news_list = newsapi.get_everything(sources=source,
                                                       language='en',
                                                       from_param= from_date,
                                                       to = to_date,
                                                       page_size=100,
                                                       page=i)

                    articles = news_list['articles']
                    temp = pd.DataFrame(articles)
                    temp['target'] = leanings
                    temp['source'] = temp['source'].map(lambda x: x['id'])
                    temp['publishedAt'] = pd.to_datetime(temp['publishedAt'])
                    temp['month'] = temp['publishedAt'].map(lambda x: x.month)
                    data = pd.concat([data,temp])
                except:
                    pass
        except:
            pass
    
    data = data.reset_index(drop=True)
    
    return data

In [9]:
add_far_left = generate_news(sources = far_left, newsapi = master, 
                            from_date = '2020-03-01', to_date = '2020-09-31',
                            leanings = 'far_left', dataframe = None)

no provided dataframe
looking for buzzfeed
looking for mashable
looking for msnbc
looking for new-york-magazine
looking for vice-news


In [10]:
add_left = generate_news(sources = left, newsapi = master, 
                            from_date = '2020-03-01', to_date = '2020-09-31',
                            leanings = 'left', dataframe = add_far_left)

provided dataframe
looking for abc-news
looking for al-jazeera-english
looking for bloomberg
looking for business-insider
looking for cbs-news
looking for cnn
looking for google-news
looking for independent
looking for nbc-news
looking for politico
looking for time


In [11]:
add_center = generate_news(sources = center, newsapi = master, 
                            from_date = '2020-03-01', to_date = '2020-09-31',
                            leanings = 'center', dataframe = add_left)

provided dataframe
looking for associated-press
looking for axios
looking for bbc-news
looking for national-geographic
looking for reuters
looking for business-insider
looking for fortune
looking for the-hill


In [12]:
add_right = generate_news(sources = right, newsapi = master, 
                            from_date = '2020-03-01', to_date = '2020-09-31',
                            leanings = 'right', dataframe = add_center)

provided dataframe
looking for fox-news
looking for the-american-conservative
looking for the-washington-times
looking for the-wall-street-journal


In [13]:
add_far_right = generate_news(sources = far_right, newsapi = master, 
                            from_date = '2020-03-01', to_date = '2020-09-31',
                            leanings = 'far_right', dataframe = add_right)

provided dataframe
looking for breitbart-news
looking for national-review


In [14]:
final_df = add_far_right

In [15]:
final_df.target.value_counts()

left         105801
center        62264
far_left      41117
far_right     18783
right         17511
Name: target, dtype: int64

In [16]:
final_df.shape

(245476, 10)

In [18]:
final_df.head()

,source,author,title,description,url,urlToImage,publishedAt,content,target,month
0,buzzfeed,"[{""@type"":""Person"",""name"":""Morgan Sloss"",""url""...",Peacock Apologized For Making Fun Of Selena Go...,"""This is so disgusting and unnecessary.""View E...",https://www.buzzfeed.com/morgansloss1/selena-g...,https://img.buzzfeed.com/buzzfeed-static/stati...,2020-11-29 17:52:13+00:00,Instagram: @selenagomezIn the caption of this ...,far_left,11.0
1,buzzfeed,"[{""@type"":""Person"",""name"":""Stephen LaConte"",""u...",Former Participants On Home Makeover Shows Are...,"""In reality, it was literally things stuck tog...",https://www.buzzfeed.com/stephenlaconte/home-m...,https://img.buzzfeed.com/buzzfeed-static/stati...,2020-11-29 17:52:13+00:00,FIL is a contractor. His team helped with a bu...,far_left,11.0
2,buzzfeed,"[{""@type"":""Person"",""name"":""Ryan Schocket"",""url...",I Love Camila Cabello's Instagram About What I...,"""It’s not as simple as it looks in pictures so...",https://www.buzzfeed.com/ryanschocket2/camila-...,https://img.buzzfeed.com/buzzfeed-static/stati...,2020-11-29 17:50:38+00:00,"""Its not as simple as it looks in pictures som...",far_left,11.0
3,buzzfeed,"[{""@type"":""Person"",""name"":""Ryan Schocket"",""url...",17 Hilarious Tweets From This Weekend,"""Look who decided to come out of their room.""V...",https://www.buzzfeed.com/ryanschocket2/tweets-...,https://img.buzzfeed.com/buzzfeed-static/stati...,2020-11-29 17:36:16+00:00,None,far_left,11.0
4,buzzfeed,"[{""@type"":""Person"",""name"":""Audrey Engvalson"",""...",You're Deeply Affectionate If You've Called Yo...,Your boo? Babe? Honey? Snookums? Sugar booger?...,https://www.buzzfeed.com/audreyworboys/pet-nam...,https://img.buzzfeed.com/buzzfeed-static/stati...,2020-11-29 17:20:11+00:00,Your boo? Babe? Honey? Snookums? Sugar booger?,far_left,11.0


In [21]:
final_df.author[0]

'[{"@type":"Person","name":"Morgan Sloss","url":"https://www.buzzfeed.com/morgansloss1","jobTitle":"BuzzFeed Staff"}]'

In [24]:
def get_author_name(value):
    try:
        eval_val = eval(value)[0]
        name = eval_val['name']
        return name
    except:
        return 'error'
    

In [25]:
def get_author_title(value):
    try:
        eval_val = eval(value)[0]
        title = eval_val['jobTitle']
        return title
    except:
        return 'error'

In [27]:
final_df['author_name'] = final_df['author'].apply(get_author_name)

In [28]:
final_df['author_title'] = final_df['author'].apply(get_author_title)

In [46]:
list(final_df.author_name)

['Morgan Sloss',
 'Stephen LaConte',
 'Ryan Schocket',
 'Ryan Schocket',
 'Audrey Engvalson',
 'Liz Richardson',
 'Brian Galindo',
 'Natasha Jokic',
 'Chantel Turner',
 'Terry Carter Jr.',
 'Shelby Heinrich',
 'Hope Lasater',
 'Chelsea Stuart',
 'Sam Cleal',
 'Samantha Wieder',
 'Kristen Harris',
 'Sumedha Bharpilania',
 'Chelsea Stuart',
 'Jasmine Vaughn-Hall',
 'Daniella Emanuel',
 'Genevieve Scarano',
 'Mallory Mower',
 'AnaMaria Glavan',
 'Hope Lasater',
 'Kayla Suazo',
 'Angelica Martinez',
 'Hannah Marder',
 'Tessa Fahey',
 'Matt Stopera',
 'Rachel Dunkel',
 'Andy Golder',
 'Morgan Sloss',
 'Kayla Yandoli',
 'Andrew Ziegler',
 'Anna Kopsky',
 'Kayla Yandoli',
 'Jasmine Vaughn-Hall',
 'HOLYMUFFINS',
 'Marquaysa Battle',
 'Audrey Engvalson',
 'Taylor Steele',
 'Michele Bird',
 'Asia McLain',
 'Christine Forbes',
 'Ryan Schocket',
 'Andria Moore',
 'Natasha Jokic',
 'Terry Carter Jr.',
 'Jonathan Mazzei',
 'Valeza Bakolli',
 'Brian Galindo',
 'Natasha Jokic',
 'TateWrites',
 'Kayla 

In [35]:
final_df.to_csv("big_data_file.csv")

In [21]:
final_df['publishedAt'] = pd.to_datetime(final_df['publishedAt'])

In [22]:
final_df['month'] = final_df['publishedAt'].map(lambda x: x.month)

In [23]:
final_df.month.value_counts()

4     10521
3      9727
5      8418
10     7979
7      6115
9      5882
8      5518
6      5029
11      686
Name: month, dtype: int64

In [24]:
final_df.source.value_counts()

business-insider             4800
politico                     2400
independent                  2400
cbs-news                     2400
abc-news                     2400
bbc-news                     2400
associated-press             2400
fox-news                     2400
breitbart-news               2400
msnbc                        2400
google-news                  2400
the-hill                     2400
vice-news                    2400
the-wall-street-journal      2400
nbc-news                     2400
al-jazeera-english           2400
time                         2400
bloomberg                    2400
cnn                          2400
reuters                      2400
axios                        2400
mashable                     2090
fortune                      1524
new-york-magazine            1257
national-review              1091
buzzfeed                      623
the-american-conservative     300
the-washington-times          172
national-geographic            18
Name: source, 

In [4]:
def get_full_text(url):
    try:
        article = Article(url)
        article.download()
        article.parse()
        article.nlp()

        title = article.title
        text = article.text
        full_text = article.title + " " + article.text
        
        return full_text
    
    except:
        return '401 Error'

In [45]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245476 entries, 0 to 245475
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   source        245476 non-null  object 
 1   author        220308 non-null  object 
 2   title         245373 non-null  object 
 3   description   237582 non-null  object 
 4   url           245476 non-null  object 
 5   urlToImage    223240 non-null  object 
 6   publishedAt   245476 non-null  object 
 7   content       184616 non-null  object 
 8   target        245476 non-null  object 
 9   month         245476 non-null  float64
 10  author_name   245476 non-null  object 
 11  author_title  245476 non-null  object 
dtypes: float64(1), object(11)
memory usage: 22.5+ MB


In [44]:
for row in final_df.iterrows():
    print(row[1].url)
    break

https://www.buzzfeed.com/morgansloss1/selena-gomez-kidney-jokes-saved-by-the-bell


In [5]:
get_full_text('https://www.bbc.com/news/health-54873105')

'Covid vaccine: First \'milestone\' vaccine offers 90% protection Covid vaccine: First \'milestone\' vaccine offers 90% protection By James Gallagher\n\nHealth and science correspondent Published duration 1 day ago Related Topics Coronavirus pandemic\n\nimage copyright Getty Images\n\nThe first effective coronavirus vaccine can prevent more than 90% of people from getting Covid-19, a preliminary analysis shows.\n\nThe developers - Pfizer and BioNTech - described it as a "great day for science and humanity".\n\nTheir vaccine has been tested on 43,500 people in six countries and no safety concerns have been raised.\n\nThe companies plan to apply for emergency approval to use the vaccine by the end of the month.\n\nNo vaccine has gone from the drawing board to being proven highly effective in such a short period of time.\n\nThere are still huge challenges ahead, but the announcement has been warmly welcomed with scientists describing themselves smiling "ear to ear" and some suggesting lif

In [26]:
def get_full_text(url):
    try:
        article = Article(url)
        article.download()
        article.parse()
        article.nlp()

        return article.text
    
    except:
        return '401 Error'

In [28]:
from tqdm import tqdm
tqdm.pandas()

/Users/alecmccabe/opt/anaconda3/lib/python3.8/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [29]:
final_df.target.value_counts()

left         26400
center       15942
far_left      8770
right         5272
far_right     3491
Name: target, dtype: int64

**Using Newspaper3k library to scrape full article content from NewsApi URLs**
- Please notice that this took over 14 hours to run

In [30]:
final_df['full_text'] = final_df['url'].progress_apply(get_full_text)

 42%|████▏     | 25179/59875 [6:20:28<20:23:19,  2.12s/it]/Users/alecmccabe/opt/anaconda3/lib/python3.8/site-packages/dateutil/parser/_parser.py:1213: UnknownTimezoneWarning: tzname IST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
100%|██████████| 59875/59875 [14:46:32<00:00,  1.13it/s]   


In [32]:
final_df.to_csv('initial_data2.csv')